# Chapter 6 데이터 랭글링

## 1. `readr` 패키지

* `tibble`로 외부 파일을 불러오거나 저장하는 함수를 제공하는 패키지

|**함수**|**설명**|
|:-:|:-:|
|`read_csv()`||
|`read_tsv()`||
|`read_delim()`|임의의 구분자로 구분된 파일 불러오기|
|`read_table()`|공백으로 구분된 파일 불러오기|
|`read_spss()`|(haven 패키지)|
|`read_sas()`|(haven 패키지)|

* `read_csv()` 함수

> * `read.csv()` 함수에 비해 약 10배 정도 빠르며, 좀 더 재현성이 뛰어남
> * 첫 번째 행은 변수 이름(header)으로 사용
> * 매개변수 `col_names` - 변수명(header)이 없는 경우 가용
>   * `col_names = FALSE` - 변수명을 X1부터 Xn까지 순차적으로 넣어줌
>   * `col_names = c("x", "y", "z")`
> * 매개변수 `skip` - 파일의 첫 n행 생략 (유효한 데이터가 n줄 이후부터 있는 경우)
> * 매개변수 `na` - 특정 값을 결측값으로 지정

```R
read_csv(directory, col_names = c("A1", "A2", ..., "An"), skip = 5, na = "9999" )
```

## 2. `dplyr` 패키지

* 데이터 가공, EDA 등에 매우 유용하고 편리한 함수 제공하는 패키지
* 적용 대상에 따른 `dplyr` 함수
    * 행 : `filter(), arrange()`
    * 열 : `select(), mutate(), rename(), relocate()`
    * 집단 : `summarise(), group_by()`

|변수|설명|
|:-:|:-:|
|year|출발년도|
|month|출발 월|
|day|출발 일|
|dep_time|실제 출발 시간(HHMM 또는 HMM)|
|arr_time|실제 도착 시간(HHMM 또는 HMM)|
|sched_dep_time|예정된 출발 시간(HHMM 또는 HMM)|
|sched_arr_time|예정된 도착 시간(HHMM 또는 HMM)|
|dep_delay|출발 지연 시간(분)|
|arr_delay|도착 지연 시간(분)|
|carrier|항공사 약어|
|flight|항공편 번호|
|tailnum|비행기 식별 번호|
|origin|출발지|
|dest|목적지|
|air_time|비행 시간(분)|
|distance|비행 거리(miles)|
|hour|예정된 출발 시간; 시|
|minute|예정된 출발 시간; 분|
|time_hour|예정된 비행 날짜 및 시간(POSIXct date)|

> `str` 대신 `glimpse`를 사용하여 콘솔 크기에 맞춰 출력할 수 있음

`-` `filter()` 함수

```R
library(nycflights13)

#### 120분 이상 연착한 항공편 - arr_delay가 120분 이상인 샘플
flights %>% filter(arr_delay >= 120)

#### 다중 조건 : 1월 1일에 출발한 항공편 - month는 1, day는 1인 샘플
flights %>% filter((month == 1) & (day == 1)) ## 832개
flights %>% filter(month == 1, day == 1) ## 여러 개의 args 입력하면 조건으로 인식함

#### 여러 값 추출 : 3, 5, 8월에 출발한 항공권 -> %in% 사용
flights %>% filter(month %in% c(3, 5, 8))
```

`-` `arrange()` 함수 : 데이터프레임 정렬 수행

> 결측치는 항상 후순위로 정렬
>
> `desc()` : 내림차순 정렬

```R
#### 출발 시간 순으로 정렬
flights %>% arrange(year, month, day, dep_time)

#### 도착 지연 순으로 내림차순 정렬
flights %>% arrange(desc(arr_delay))
```

`-` `select()` 함수 : DB에서 쓰는 `SELECT`와 동일

```R
flights %>% select(x1, x2, x3) ## 특정 변수들 선택
flights %>% select(x1:x5)  ## x1부터 x5까지 변수 이름으로 슬라이싱
flights %>% select(-(x1:x5)) ## x1부터 x5까지 빼고 추출

#### 추출 시 변수 이름 변경까지 됨
flights %>%
  select(dep.time = dep_time,
         arr_time = arr_time)
```

`-` 도우미(helpers) 함수

|함수|설명|
|:-:|:-:|
|`starts_with()`|특정 문자열로 시작하는 이름 매칭|
|`ends_with()`|특정 문자열로 끝나는 이름 매칭|
|`contains()`|특정 문자열을 포함하는 이름 매칭|
|`num_range(perfix, range)`|숫자 접미사를 가지는 일련의 이름 매칭|

```R
### select 도우미 함수
flights %>% select(starts_with("sched")) ## 변수 이름이 "sched"로 시작하는 경우
flights %>% select(ends_with("time"))    ## 변수 이름이 "time"으로 끝나는 경우

flights %>% select(contains("arr"))         ## 변수 이름에 "arr"이 포함되는 경우
## -> carrier라는 의도하지 않은 변수가 들어감
flights %>% select(contains("arr_")) ## 이런 식으로 응용 가능하긴 한데, 좀 그럼
```

`-` `mutate()` 함수

`-` `rename()` 함수

* 특정 변수 이름 변경
* 명시적으로 언급하지 않은 모든 변수를 유지함(백틱까지 쓰면서)
* 숫자열이 앞에 들어온 변수명 같이 변수 이름 규칙을 위반하는 것을 교체하는 데에 유리

```R
#### select에서와 동일한 사용법
flights %>% rename(dep.time = dep_time,
                   arr.time = arr_time)
```

`-` `relocate()` 함수 : 변수의 위치 변경

```R
flights %>%
  filter(dep_delay <= 10 & dep_delay >= -10) %>%
  arrange(desc(arr_delay)) %>%
  relocate(arr_delay, .after = day) ## 앞에 day를 둠

flights %>%
  filter(dep_delay <= 10 & dep_delay >= -10) %>%
  arrange(desc(arr_delay)) %>%
  relocate(arr_delay, .before = day) ## 뒤에 day를 둠

flights %>% relocate(air_time, distance) ## 기본적으로 앞에 데려옴
flights %>% relocate(carrier:tailnum, .before = day)  ## 콜론 사용 문법은 공유
```